In [45]:
class Scalar:
    def __init__(self, data, grad = 0):
        self.data = data
        self.grad = grad
    def __repr__(self):
        return f"Scalar(Value: {self.data} Grad: {self.grad})"
    def __add__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data + other.data)
        return out
    def __radd__(self, other):
        return self + other
    def __mul__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data * other.data)
        return out
    def __rmul__(self, other):
        return self * other
    def __pow__(self, other):
        other = other if isinstance(other, Scalar) else Scalar(other)
        out = Scalar(self.data ** other.data)
        return out
    def __rpow__(self, other):
        return self ** other
    def exp(self):
        return 2.718281828459045 ** self
        

In [46]:
a = Scalar(2)
b = 3
a.exp()

Scalar(Value: 6.5808859910179205 Grad: 0)